Upconversion es un fenómeno en el que fotones de una determinada energía excitan múltiples veces a un mismo electrón llevandolo a un estado de mayor energía que la energía individual de los fotones que incidieron.
Si este electrón luego decae al estado fundamental, emite un fotón de menor longitud de onda que los incidentes.
Hay muchos mecanismos de upconversion, pero los principales son Energy Transfer Upconversion (ETU) y Excited State Absorption (ESA). 
En este diagrama se ejemplifican los mecanismos:

![image](images/etu-esa.png "Figura 1")

Por un lado, ETU consiste en la interacción dipolo-dipolo de dos iones en el estado excitado N1, que lleva a que un tercer ión (o uno de los dos que ya estaba excitado) al estado excitado de doble energía N2, llevando a los dos iones que interactuaron nuevamente al estado fundamental.

Por otro lado, ESA consiste en la excitación de un ión en el estado N1 por un fotón con la energía necesaria para llevarlo al estado N2.

Se propone entonces estudiar este sistema bajo las siguientes hipótesis:
1. La densidad de población del estado fundamental es constante
1. Se excita al sistema con onda continua
1. La upconversion está dada por ETU o ESA.
1. Los estados excitados $N_i$ tienen tiempos de vida $\tau_i$ y decaen proporcionalmente a $A_i = \tau_i^{-1}$.

Bajo algunas aproximaciones que se pueden leer en el [pollnau](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.61.3337), la densidad de potencia incidente por el láser está dada por 

$$\rho_P = \frac{\lambda_P}{hc\pi w_P^2}P$$

Donde $\lambda_P$ es la longitud de onda de excitación, $h$ la constante de Plank, $c$ la velocidad de la luz, $w_P$ el radio de bombeo y $P$ la potencia incidente.
Entonces, las rate equations para la excitación por fotones de un estado $N_1$ a otro $N_2$ por un fotón está dada por 
$$\dot{N}_1 = - \rho_P \sigma_1 N_1$$
$$\dot{N}_2 = \rho_P \sigma_1 N_1$$
donde $\sigma_1$ es la cross section del estado $N_1$.
Lo más importante acá es que estos términos son proporcionales a la potencia incidente del láser.

Por otro lado, las ecuaciones para dos niveles que interactúan dipolarmente para generar ETU son 
$$\dot{N}_1 = - 2 W_1 N_1^2$$
$$\dot{N}_2 = W_1 N_1^2$$
donde $W_1$ es la constante del rate de ETU. En este caso, se pierden dos iones en el estado $N_1$ para ganar uno en el $N_2$ el ión restante vuelve al estado $N_0$, pero por ahora estamos asumiendo que $\dot{N}_0 = 0$ por lo que esa ecuación no nos interesa.

Entonces, para el sistema de 3 niveles con ETU y ESA las ecuaciones quedan 

![image](images/eqdif-3niv.jpeg)

Las ecuación estacionaria para $N_1$ queda

![image](images/estacionario_k1mgw1.jpeg)

Nos interesa estudiar el caso estacionario bajo algunas aproximaciones.
De esta ecuación hay que notar que el primer término es proporcional a $P^0$ y el segundo a $P^{-1}$.

##### $k_1 >> W_1 N_1$

Bajo esta aproximación se va el segundo término y queda $N_1 \propto P^0$ para potencias altas y $N_1 \propto P$ para potencias bajas.
Ahora, qué pasa con $N_2$ en cada caso?
$P>>1  \implies N_2 \propto P$, y $P<<1 \implies N_2 \propto P^2$.
Esto quiere decir que en un gráfico loglog de intensidad vs potencia, la pendiente de $N_2$ sería 1 para potencias altas y 2 para bajas.
Para verificarlo hago una simulación del sistema con las ecuaciones que escribí antes.



In [2]:
from poincare import Simulator
from nb_tesis.nb_tesis_1 import Pollnau, p_vs_i
import numpy as np
import matplotlib.pyplot as plt


sim = Simulator(Pollnau)
kwargs = {
    "model_params":{
        Pollnau.N0:1,
        Pollnau.N1:0,
        Pollnau.N2:0,
        Pollnau.P:1,
        Pollnau.sigma0:1,
        Pollnau.sigma1:1e0,
        Pollnau.k1:1e-1,
        Pollnau.k2:1e0,
        Pollnau.W1:1e1
    },
    "save_at":np.linspace(0, 10, 100)
}